In [ ]:
# Cell 1: Introduction and Setup
"""
# 🤖 Multi-Model LLM Comparison Tool
## Google Colab Tutorial

Learn how to compare responses from multiple AI models and generate professional PDF reports!

What you'll learn:
- Connect to OpenRouter API (access to many AI models)
- Query multiple models simultaneously
- Compare responses side by side
- Generate professional PDF reports
"""

print("🎓 Welcome to AI Model Comparison with PDF Reports!")
print("Let's compare AI models and create professional documentation!")


In [ ]:
# Cell 2: Install and Import Everything
# Install required packages
!pip install requests reportlab

# Import all required libraries
import requests
import json
import time
import os
import html
import textwrap
from datetime import datetime
from typing import List, Dict

# ReportLab imports for PDF generation
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib import colors
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_JUSTIFY

print("✅ All packages installed and imported!")


In [ ]:
# Cell 3: Complete AI Comparison Class
class AIModelComparison:
    def __init__(self, api_key):
        """Initialize with your OpenRouter API key"""
        self.api_key = api_key
        self.base_url = "https://openrouter.ai/api/v1/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        # Popular models that work well
        self.models = [
            "anthropic/claude-3.5-sonnet",
            "meta-llama/llama-3.1-70b-instruct",
            "openai/gpt-4o-mini",
            "meta-llama/llama-2-70b-chat",
            "google/gemini-flash-1.5",
            "deepseek/deepseek-r1:free"
        ]
        
    def query_model(self, model: str, prompt: str, max_tokens: int = 1500) -> Dict:
        """Query a specific model"""
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": max_tokens,
            "temperature": 0.7
        }
        
        try:
            response = requests.post(self.base_url, headers=self.headers, json=payload, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                return {
                    "model": model,
                    "success": True,
                    "response": data["choices"][0]["message"]["content"].strip(),
                    "tokens_used": data.get("usage", {}).get("total_tokens", "N/A"),
                    "error": None
                }
            else:
                return {
                    "model": model,
                    "success": False,
                    "response": f"Error: {response.status_code}",
                    "tokens_used": "N/A",
                    "error": response.text
                }
                
        except Exception as e:
            return {
                "model": model,
                "success": False,
                "response": f"Exception: {str(e)}",
                "tokens_used": "N/A",
                "error": str(e)
            }
    
    def query_multiple_models(self, prompt: str, max_tokens: int = 1500, delay: float = 0.5) -> List[Dict]:
        """Query multiple models with the same prompt"""
        responses = []
        
        print(f"🤖 Querying {len(self.models)} models...")
        print(f"📝 Prompt: {prompt[:100]}...")
        print("=" * 80)
        
        for i, model in enumerate(self.models, 1):
            print(f"[{i}/{len(self.models)}] Querying {model}...", end=" ")
            
            response = self.query_model(model, prompt, max_tokens)
            responses.append(response)
            
            if response["success"]:
                print("✅")
            else:
                print("❌")
            
            # Add delay to avoid rate limiting
            if i < len(self.models):
                time.sleep(delay)
        
        return responses
    
    def display_responses(self, responses: List[Dict], prompt: str):
        """Display responses in a nice readable format"""
        print("\n" + "=" * 120)
        print(f"🕒 RESULTS - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 120)
        print(f"📝 ORIGINAL PROMPT: {prompt}")
        print("=" * 120)
        
        # Calculate column width
        col_width = max(35, max(len(r["model"]) for r in responses) + 5)
        
        # Header
        header = f"{'MODEL':<{col_width}} | {'STATUS':<8} | {'TOKENS':<8} | RESPONSE"
        print(header)
        print("-" * len(header))
        
        for response in responses:
            model_name = response["model"]
            status = "✅ OK" if response["success"] else "❌ FAIL"
            tokens = str(response["tokens_used"])
            resp_text = response["response"]
            
            # Truncate long responses for table view
            if len(resp_text) > 80:
                resp_preview = resp_text[:77] + "..."
            else:
                resp_preview = resp_text
            
            # Replace newlines with spaces for table format
            resp_preview = resp_preview.replace('\n', ' ').replace('\r', ' ')
            
            print(f"{model_name:<{col_width}} | {status:<8} | {tokens:<8} | {resp_preview}")
        
        print("=" * 120)
        
        # Detailed responses
        print("\n📋 DETAILED RESPONSES:")
        print("=" * 120)
        
        for i, response in enumerate(responses, 1):
            print(f"\n[{i}] 🤖 MODEL: {response['model']}")
            print(f"📊 STATUS: {'✅ Success' if response['success'] else '❌ Failed'}")
            print(f"🔢 TOKENS: {response['tokens_used']}")
            print("📄 RESPONSE:")
            print("-" * 60)
            print(response["response"])
            print("-" * 60)

print("✅ AIModelComparison class ready!")


In [ ]:
# Cell 4: PDF Generation Methods
# Enhanced PDF Generation - Full 2-Column Layout

def save_to_pdf_columns(self, responses: List[Dict], prompt: str, filename: str = None):
    """Save responses to PDF with 2-column layout - FIXED VERSION"""
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"openrouter_responses_{timestamp}.pdf"
    
    def clean_text_for_pdf(text: str) -> str:
        """Robust text cleaning for PDF generation"""
        if not text:
            return "No content available"
        
        # Convert to string
        text = str(text)
        
        # Remove null bytes and other problematic control characters
        text = text.replace('\x00', '').replace('\x08', '').replace('\x0b', '').replace('\x0c', '')
        text = text.replace('\x0e', '').replace('\x0f', '').replace('\x10', '').replace('\x11', '')
        
        # Replace problematic characters with safe alternatives
        replacements = {
            '&': 'and',
            '<': '[',
            '>': ']',
            '"': "'",
            '\\': '/',
            '\u2018': "'",  # Left single quote
            '\u2019': "'",  # Right single quote
            '\u201c': '"',  # Left double quote
            '\u201d': '"',  # Right double quote
            '\u2013': '-',  # En dash
            '\u2014': '-',  # Em dash
            '\u2026': '...',  # Ellipsis
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Replace newlines with space for table cells (ReportLab handles line breaks differently)
        text = text.replace('\n', ' ').replace('\r', ' ')
        
        # Remove non-printable characters but keep basic punctuation
        text = ''.join(char for char in text if ord(char) >= 32 and ord(char) <= 126)
        
        # Limit length to prevent table overflow
        if len(text) > 3000:
            text = text[:3000] + "... [truncated for PDF]"
        
        return text.strip() or "No content"
    
    try:
        print(f"Creating PDF: {filename}")
        
        # Create PDF document
        doc = SimpleDocTemplate(
            filename,
            pagesize=A4,
            rightMargin=40,
            leftMargin=40,
            topMargin=50,
            bottomMargin=50
        )
        
        story = []
        styles = getSampleStyleSheet()
        
        # Custom styles - safer approach
        title_style = ParagraphStyle(
            'SafeTitle',
            parent=styles['Title'],
            fontSize=18,
            spaceAfter=20,
            alignment=TA_CENTER,
            textColor=colors.navy
        )
        
        heading_style = ParagraphStyle(
            'SafeHeading',
            parent=styles['Heading2'],
            fontSize=12,
            spaceAfter=10,
            spaceBefore=15,
            textColor=colors.navy
        )
        
        model_style = ParagraphStyle(
            'SafeModel',
            parent=styles['Normal'],
            fontSize=9,
            textColor=colors.navy,
            alignment=TA_LEFT,
            fontName='Helvetica-Bold'
        )
        
        response_style = ParagraphStyle(
            'SafeResponse',
            parent=styles['Normal'],
            fontSize=8,
            alignment=TA_LEFT,
            leftIndent=3,
            rightIndent=3,
            spaceAfter=2
        )
        
        # Title
        story.append(Paragraph("Multi-Model AI Comparison Report", title_style))
        story.append(Spacer(1, 12))
        
        # Metadata
        story.append(Paragraph(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", styles['Normal']))
        story.append(Paragraph(f"Total Models: {len(responses)}", styles['Normal']))
        
        successful = sum(1 for r in responses if r["success"])
        story.append(Paragraph(f"Successful Responses: {successful}/{len(responses)}", styles['Normal']))
        story.append(Spacer(1, 15))
        
        # Original prompt
        story.append(Paragraph("Original Prompt", heading_style))
        clean_prompt = clean_text_for_pdf(prompt)
        story.append(Paragraph(clean_prompt, styles['Normal']))
        story.append(Spacer(1, 20))
        
        # Responses section
        story.append(Paragraph("Model Responses", heading_style))
        
        # Prepare table data with safer formatting
        table_data = []
        
        # Header row
        table_data.append([
            Paragraph("Model Name", model_style),
            Paragraph("Response", model_style)
        ])
        
        # Data rows
        for response in responses:
            # Model name with status (using text symbols instead of emojis)
            model_name = response["model"].split('/')[-1]
            status_symbol = "[OK]" if response["success"] else "[FAIL]"
            tokens = f"({response['tokens_used']} tokens)" if response["success"] and response["tokens_used"] != "N/A" else ""
            
            # Safe model cell content
            model_cell_text = f"{model_name} {status_symbol} {tokens}"
            model_cell_clean = clean_text_for_pdf(model_cell_text)
            
            # Response content
            if response["success"]:
                response_text = clean_text_for_pdf(response["response"])
            else:
                error_text = response.get("error", response["response"])
                response_text = f"Error: {clean_text_for_pdf(error_text)}"
            
            # Add to table
            table_data.append([
                Paragraph(model_cell_clean, model_style),
                Paragraph(response_text, response_style)
            ])
        
        # Create table with proper column widths
        col_widths = [2.2*inch, 4.8*inch]  # Adjusted for A4
        
        comparison_table = Table(table_data, colWidths=col_widths, repeatRows=1)
        
        # Simplified table styling
        table_style = [
            # Header
            ('BACKGROUND', (0, 0), (-1, 0), colors.navy),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
            ('TOPPADDING', (0, 0), (-1, 0), 8),
            
            # Data cells
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
            ('ALIGN', (0, 1), (-1, -1), 'LEFT'),
            ('VALIGN', (0, 1), (-1, -1), 'TOP'),
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 8),
            ('LEFTPADDING', (0, 1), (-1, -1), 6),
            ('RIGHTPADDING', (0, 1), (-1, -1), 6),
            ('TOPPADDING', (0, 1), (-1, -1), 8),
            ('BOTTOMPADDING', (0, 1), (-1, -1), 8),
            
            # Borders
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
            ('LINEBELOW', (0, 0), (-1, 0), 2, colors.navy),
        ]
        
        # Add alternating row colors
        for i in range(1, len(table_data)):
            if i % 2 == 0:
                table_style.append(('BACKGROUND', (0, i), (-1, i), colors.lightgrey))
        
        comparison_table.setStyle(TableStyle(table_style))
        
        # Add table to story
        story.append(comparison_table)
        story.append(Spacer(1, 20))
        
        # Footer
        footer_text = f"Report generated on {datetime.now().strftime('%Y-%m-%d at %H:%M:%S')}"
        story.append(Paragraph(footer_text, styles['Normal']))
        
        # Build the PDF
        doc.build(story)
        print(f"PDF successfully created: {filename}")
        print("Features:")
        print("  - 2-column layout (Model | Response)")
        print("  - Full responses included")
        print("  - Professional table formatting")
        print("  - Status indicators and token counts")
        print("  - Robust character handling")
        return filename
        
    except Exception as e:
        print(f"Error creating PDF: {str(e)}")
        print(f"Error type: {type(e).__name__}")
        
        # Try to provide more specific error information
        import traceback
        print("Detailed error:")
        traceback.print_exc()
        
        return None

# Replace the PDF method with the fixed version
AIModelComparison.save_to_pdf = save_to_pdf_columns

print("Fixed 2-Column PDF Generator Ready!")
print("Improvements:")
print("  - Robust character cleaning (no emojis)")
print("  - Better error handling and debugging")
print("  - Text length limits to prevent overflow") 
print("  - Safer HTML formatting")
print("  - ASCII-only character set")
print("  - Improved table sizing for A4 pages")


def save_to_pdf_columns(self, responses: List[Dict], prompt: str, filename: str = None):
    """Save responses to PDF with 2-column layout: Model Name | Full Response"""
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"openrouter_responses_{timestamp}.pdf"
    
    def clean_text_for_pdf(text: str) -> str:
        """Clean text for PDF while preserving formatting"""
        if not text:
            return ""
        
        # Convert to string and basic cleanup
        text = str(text).replace('\x00', '').replace('\x08', '').replace('\x0b', '').replace('\x0c', '')
        
        # Handle common problematic characters for ReportLab
        text = text.replace('&', '&amp;')
        text = text.replace('<', '&lt;')
        text = text.replace('>', '&gt;')
        
        # Preserve line breaks and formatting
        text = text.replace('\n', '<br/>')
        
        # Remove only truly problematic characters
        text = ''.join(char for char in text if ord(char) >= 32 or char in '\n\r\t')
        
        return text
    
    try:
        # Create PDF with wider margins for better table layout
        doc = SimpleDocTemplate(
            filename,
            pagesize=A4,
            rightMargin=36,  # Smaller margins for more space
            leftMargin=36,
            topMargin=50,
            bottomMargin=50
        )
        
        story = []
        styles = getSampleStyleSheet()
        
        # Custom styles
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Title'],
            fontSize=20,
            spaceAfter=20,
            alignment=TA_CENTER,
            textColor=colors.darkblue
        )
        
        heading_style = ParagraphStyle(
            'CustomHeading',
            parent=styles['Heading2'],
            fontSize=14,
            spaceAfter=10,
            spaceBefore=15,
            textColor=colors.darkblue
        )
        
        # Style for model names in table
        model_style = ParagraphStyle(
            'ModelStyle',
            parent=styles['Normal'],
            fontSize=10,
            textColor=colors.darkblue,
            alignment=TA_LEFT,
            fontName='Helvetica-Bold'
        )
        
        # Style for responses in table
        response_style = ParagraphStyle(
            'ResponseStyle',
            parent=styles['Normal'],
            fontSize=9,
            alignment=TA_JUSTIFY,
            leftIndent=5,
            rightIndent=5,
            spaceAfter=3
        )
        
        # Title and header info
        story.append(Paragraph("Multi-Model LLM Comparison Report", title_style))
        story.append(Spacer(1, 10))
        
        # Metadata
        story.append(Paragraph(f"<b>Generated:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", styles['Normal']))
        story.append(Paragraph(f"<b>Total Models:</b> {len(responses)}", styles['Normal']))
        
        successful = sum(1 for r in responses if r["success"])
        story.append(Paragraph(f"<b>Successful Responses:</b> {successful}/{len(responses)}", styles['Normal']))
        story.append(Spacer(1, 15))
        
        # Original prompt
        story.append(Paragraph("Original Prompt", heading_style))
        clean_prompt = clean_text_for_pdf(prompt)
        story.append(Paragraph(clean_prompt, response_style))
        story.append(Spacer(1, 20))
        
        # Create the main comparison table
        story.append(Paragraph("Model Responses Comparison", heading_style))
        
        # Prepare table data
        table_data = []
        
        # Header row
        table_data.append([
            Paragraph('<b>Model Name</b>', model_style),
            Paragraph('<b>Response</b>', model_style)
        ])
        
        # Data rows - each model and its full response
        for response in responses:
            # Model name with status
            model_name = response["model"].split('/')[-1]  # Get just the model name
            status_icon = "✅" if response["success"] else "❌"
            tokens = f"({response['tokens_used']} tokens)" if response["success"] else ""
            
            model_cell = f"<b>{model_name}</b><br/>{status_icon} {tokens}"
            
            # Full response (no truncation)
            if response["success"]:
                full_response = clean_text_for_pdf(response["response"])
            else:
                full_response = f"<i>Error:</i> {clean_text_for_pdf(response['response'])}"
            
            # Add row to table
            table_data.append([
                Paragraph(model_cell, model_style),
                Paragraph(full_response, response_style)
            ])
        
        # Create table with appropriate column widths
        # Model name column: 2.5 inches, Response column: 5 inches
        col_widths = [2.5*inch, 5*inch]
        
        comparison_table = Table(table_data, colWidths=col_widths, repeatRows=1)
        
        # Style the table
        table_style = [
            # Header row styling
            ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 11),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
            ('TOPPADDING', (0, 0), (-1, 0), 10),
            
            # Data rows styling
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
            ('ALIGN', (0, 1), (-1, -1), 'LEFT'),
            ('VALIGN', (0, 1), (-1, -1), 'TOP'),  # Align to top
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('LEFTPADDING', (0, 1), (-1, -1), 8),
            ('RIGHTPADDING', (0, 1), (-1, -1), 8),
            ('TOPPADDING', (0, 1), (-1, -1), 10),
            ('BOTTOMPADDING', (0, 1), (-1, -1), 10),
            
            # Grid and borders
            ('GRID', (0, 0), (-1, -1), 1, colors.grey),
            ('LINEBELOW', (0, 0), (-1, 0), 2, colors.darkblue),
            
            # Alternating row colors for better readability
        ]
        
        # Add alternating row colors
        for i in range(1, len(table_data)):
            if i % 2 == 0:
                table_style.append(('BACKGROUND', (0, i), (-1, i), colors.lightgrey))
        
        comparison_table.setStyle(TableStyle(table_style))
        
        # Add table to story
        story.append(comparison_table)
        story.append(Spacer(1, 20))
        
        # Footer
        story.append(Paragraph(
            f"<i>Report generated by OpenRouter Multi-Model Comparison Tool on {datetime.now().strftime('%Y-%m-%d at %H:%M:%S')}</i>",
            styles['Normal']
        ))
        
        # Build the PDF
        doc.build(story)
        print(f"📄 Enhanced 2-column PDF saved: {filename}")
        print("✅ Features:")
        print("  • Clean 2-column layout (Model | Response)")
        print("  • Full responses with NO text cutoff")
        print("  • Professional table formatting")
        print("  • Status indicators and token counts")
        return filename
        
    except Exception as e:
        print(f"❌ Error creating enhanced PDF: {str(e)}")
        return None

# Replace the PDF method
AIModelComparison.save_to_pdf = save_to_pdf_columns

print("✅ Enhanced 2-Column PDF Generator Ready!")
print("🎯 Features:")
print("  📋 Clean 2-column table format")
print("  📄 Model Name | Full Response (no truncation)")
print("  ✨ Professional styling with alternating row colors")
print("  🔢 Token counts and status indicators")
print("  📊 Summary statistics at top")


In [ ]:
# Cell 5: JSON Export Method
def save_to_json(self, responses: List[Dict], prompt: str, filename: str = None):
    """Save responses to a JSON file"""
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"openrouter_responses_{timestamp}.json"
    
    data = {
        "timestamp": datetime.now().isoformat(),
        "prompt": prompt,
        "responses": responses,
        "statistics": {
            "total_models": len(responses),
            "successful_responses": sum(1 for r in responses if r["success"]),
            "failed_responses": sum(1 for r in responses if not r["success"]),
            "total_tokens": sum(int(r["tokens_used"]) for r in responses if str(r["tokens_used"]).isdigit())
        }
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f"💾 JSON saved: {filename}")
    return filename

# Add to class
AIModelComparison.save_to_json = save_to_json

print("✅ JSON export method added!")


In [ ]:
# Cell 6: Setup API Key
# Get your free API key from: https://openrouter.ai/
api_key = os.getenv('OPENROUTER_API_KEY')

if not api_key:
    api_key = input("Enter your OpenRouter API key: ").strip()

if api_key:
    print("🔑 API key configured!")
    ai = AIModelComparison(api_key)
    print(f"🚀 AI assistant ready with {len(ai.models)} models!")
else:
    print("❌ Need API key to continue. Get one from: https://openrouter.ai/")


In [ ]:
# Cell 7: Interactive Mode
def interactive_comparison_enhanced():
    """Interactive mode with enhanced 2-column PDF"""
    if 'ai' not in globals():
        print("❌ AI assistant not found")
        return
    
    print("🎮 Enhanced Interactive AI Model Comparison")
    print("📋 New Feature: 2-Column PDF (Model | Full Response)")
    print("=" * 60)
    
    user_prompt = input("Enter your question: ").strip()
    
    if not user_prompt:
        print("❌ Empty prompt!")
        return
    
    try:
        max_tokens = int(input("Max tokens (default 800): ") or 800)
    except ValueError:
        max_tokens = 800
    
    # Query all models
    print("\n🤖 Querying all models...")
    results = ai.query_multiple_models(user_prompt, max_tokens)
    
    # Display results
    ai.display_responses(results, user_prompt)
    
    # Ask about saving
    save_files = input("\n💾 Generate enhanced 2-column PDF report? (y/n): ").lower().strip()
    
    if save_files == 'y':
        pdf_file = ai.save_to_pdf(results, user_prompt)
        json_file = ai.save_to_json(results, user_prompt)
        print(f"\n✅ Reports saved:")
        print(f"  📄 PDF (2-column): {pdf_file}")
        print(f"  💾 JSON: {json_file}")
    
    print("✨ Enhanced comparison completed!")

print("🎉 Enhanced interactive mode ready!")
print("💡 Call: interactive_comparison_enhanced()")



In [ ]:
# Cell 8: Execute interactive Mode
interactive_comparison_enhanced()